In [22]:
import tensorflow as tf
import os
import scipy.io
import numpy as np

In [23]:
working_dir = os.getcwd()

file_path = os.path.join(working_dir, 'jaxpi', 'examples', 'allen_cahn','data', 'allen_cahn.mat')

data = scipy.io.loadmat(file_path)

In [39]:
# get data for model
x = tf.convert_to_tensor(data['x'].flatten()[:,None])
t = tf.convert_to_tensor(data['t'].flatten()[:,None])

usol = (np.real(data['usol']))
usol = tf.reshape(usol, shape=(1,201,512))

In [52]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(713,1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(201*512),
    tf.keras.layers.Reshape(target_shape=(201,512), name="output")
])

In [54]:
with tf.GradientTape(persistent=True) as tapex, tf.GradientTape(persistent=True) as tapet:
    tapex.watch(x)
    tapet.watch(t)

    inputs = tf.concat([t,x],axis=0)
    inputs = tf.transpose(inputs)

    y = model(inputs)

    print(y)

u_t = tapet.gradient(y,t)
u_x = tapex.gradient(y,x)

print(u_t)
print(u_x)

tf.Tensor(
[[[ 0.02176516 -0.01446987  0.00404839 ... -0.01747805 -0.02706672
   -0.00881613]
  [ 0.01673312 -0.02058491 -0.02081505 ...  0.01259358 -0.00196298
    0.01420073]
  [-0.02323391 -0.02896107 -0.02331344 ... -0.00823037  0.02475545
   -0.0118645 ]
  ...
  [ 0.02971118  0.01570613  0.0030185  ...  0.02496698 -0.00417827
   -0.01393665]
  [-0.04174316 -0.02207178  0.02110051 ... -0.0276143  -0.01930213
   -0.01846107]
  [ 0.00603866 -0.00189864  0.03136193 ...  0.02010597 -0.01396595
    0.0239065 ]]], shape=(1, 201, 512), dtype=float32)
tf.Tensor(
[[-9.55970585e-02]
 [-6.09797537e-01]
 [ 7.40509927e-02]
 [ 4.94924068e-01]
 [ 4.63950932e-01]
 [ 4.75998133e-01]
 [-3.77091244e-02]
 [ 4.26945537e-01]
 [-1.94365650e-01]
 [ 2.45798990e-01]
 [ 8.37660283e-02]
 [ 1.84391782e-01]
 [ 2.20130216e-02]
 [-5.52419603e-01]
 [ 3.47896278e-01]
 [-1.32908583e-01]
 [ 9.59278584e-01]
 [ 1.49611056e-01]
 [-2.84030199e-01]
 [ 1.37603134e-01]
 [ 7.01793194e-01]
 [ 8.77880871e-01]
 [-4.18740883e-02

In [35]:
# train
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])
model.fit(x=inputs,y=usol, epochs=100)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0050 - loss: 0.4786
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.0348 - loss: 0.4119
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.0199 - loss: 0.3199
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 0.0249 - loss: 0.2180
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.0299 - loss: 0.1276
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.0199 - loss: 0.0603
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.0149 - loss: 0.0210
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.0149 - loss: 0.0086
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.0199 - loss: 0.0165
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.0398 - loss: 0.0340
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.0597 - loss: 0.0504
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.

In [38]:
# calculate gradients
y = model(inputs)

print(y)

u_t = tapet.gradient(y, inputs[:200], unconnected_gradients='zero')
u_x = tapex.gradient(y, x, unconnected_gradients='zero')

print(u_t)

tf.Tensor(
[[[-0.9977231  -0.9904092  -0.9824423  ... -0.98266214 -0.9894482
   -1.0003458 ]
  [-0.99678034 -0.9925121  -0.9821588  ... -0.98306847 -0.99065226
   -0.99682736]
  [-0.9974867  -0.99001914 -0.9828634  ... -0.9826559  -0.99032164
   -0.9980665 ]
  ...
  [-0.9990657  -0.99747705 -0.9992055  ... -0.99851596 -0.99796855
   -0.9986106 ]
  [-0.9980468  -0.99895644 -0.9990162  ... -0.99950457 -0.9985215
   -0.9978001 ]
  [-0.99748975 -0.99689454 -0.9983936  ... -0.9975994  -0.9986589
   -0.9984282 ]]], shape=(1, 201, 512), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [99]:
model.evaluate(x=inputs, y=usol)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.6816 - loss: 1.2751e-14


[1.2750531797652963e-14, 0.6815920472145081]

In [250]:
# calculate gradient
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    forward = model(x)
    u_x = tape.gradient(forward, x, unconnected_gradients='zero')
    u_xx = tape.gradient(u_x, x, unconnected_gradients='zero')

#print(forward)
print(u_x)
print(u_xx)

tf.Tensor(
[[ 2.43167830e+00]
 [ 2.43249726e+00]
 [ 2.43330884e+00]
 [ 2.43411350e+00]
 [ 2.43491054e+00]
 [ 2.43570018e+00]
 [ 2.43648291e+00]
 [ 2.43725777e+00]
 [ 2.43802547e+00]
 [ 2.43878603e+00]
 [ 2.43953919e+00]
 [ 2.44028473e+00]
 [ 2.44102311e+00]
 [ 2.44175434e+00]
 [ 2.44247746e+00]
 [ 2.44319344e+00]
 [ 2.44390202e+00]
 [ 2.44460297e+00]
 [ 2.44529676e+00]
 [ 2.44598293e+00]
 [ 2.44666147e+00]
 [ 2.44733238e+00]
 [ 2.44799566e+00]
 [ 2.44865227e+00]
 [ 2.44930029e+00]
 [ 2.44994164e+00]
 [ 2.45057487e+00]
 [ 2.45123720e+00]
 [ 2.45185518e+00]
 [ 2.45246601e+00]
 [ 2.45306921e+00]
 [ 2.45366430e+00]
 [ 2.45425224e+00]
 [ 2.45483208e+00]
 [ 2.45540500e+00]
 [ 2.45596933e+00]
 [ 2.45652676e+00]
 [ 2.45707655e+00]
 [ 2.45761776e+00]
 [ 2.45815182e+00]
 [ 2.45867777e+00]
 [ 2.45919609e+00]
 [ 2.45970726e+00]
 [ 2.46021032e+00]
 [ 2.46070552e+00]
 [ 2.46119308e+00]
 [ 2.46167278e+00]
 [ 2.46214485e+00]
 [ 2.46260905e+00]
 [ 2.46306539e+00]
 [ 2.46351385e+00]
 [ 2.46395469e+00]
 

# advanced auto grad Examples

In [50]:
import tensorflow as tf

# Define your model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(4,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(1)
])

# Define some tensors
tensor1 = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
tensor2 = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)

# Compute model output
with tf.GradientTape(persistent=True) as tape:
    tape.watch(tensor1)
    tape.watch(tensor2)

    # Concatenate the tensors along axis 0
    concatenated_tensor = tf.concat([tensor1, tensor2], axis=0)
    concatenated_tensor = tf.reshape(concatenated_tensor, (1,4,2))
    print(concatenated_tensor.shape)
    
    # Pass the concatenated tensor as input to the model
    model_output = model(concatenated_tensor)

# Compute gradients of the model output with respect to the individual tensors
gradient_tensor1 = tape.gradient(model_output, tensor1)
gradient_tensor2 = tape.gradient(model_output, tensor2)

print("Gradient of the model output with respect to tensor1:")
print(gradient_tensor1)

print("\nGradient of the model output with respect to tensor2:")
print(gradient_tensor2)


(1, 4, 2)
Gradient of the model output with respect to tensor1:
tf.Tensor(
[[-0.17236525  0.09680016]
 [-0.10308962 -0.23470856]], shape=(2, 2), dtype=float32)

Gradient of the model output with respect to tensor2:
tf.Tensor(
[[ 0.17332584  0.00868668]
 [-0.24331066  0.11009061]], shape=(2, 2), dtype=float32)
